# 04 Optimize

In [1]:
import pandas as pd
import numpy as np
import sys
sys.path.append('../src')
from optimize import optimize_tourney

In [2]:
df = pd.read_csv('../data/opt_input.csv')
df.head()

,Season,Slot,StrongSeed,WeakSeed,Round,TeamID_Strong,TeamName_Strong,TeamID_Weak,TeamName_Weak,TeamID_Winner,...,Round_v2,comp,Key,Win_Prob,RPI_Strong,RPI_Weak,RPI_Strong_Dummy,RPI_Weak_Dummy,Win_Prob_glm,Win_Prob_gbm
0,2018,R1W1,W01,W16,1,1437,Villanova,1347,Radford,1437,...,1,1,115,0.992647,2,128,1,1,0.948936,0.968941
1,2018,R1W1,W01,W16,1,1437,Villanova,1347,Radford,1347,...,1,-1,115,0.007353,2,128,1,1,0.948936,0.968941
2,2018,R2W1,R1W1,R1W8,2,1437,Villanova,1439,Virginia Tech,1437,...,2,1,207,0.811594,2,61,1,1,0.796519,0.869192
3,2018,R2W1,R1W1,R1W8,2,1437,Villanova,1439,Virginia Tech,1439,...,2,-1,207,0.188406,2,61,1,1,0.796519,0.869192
4,2018,R2W1,R1W1,R1W8,2,1347,Radford,1439,Virginia Tech,1347,...,2,-1,208,0.254310,128,61,1,1,0.173994,0.223140


In [3]:
# weight win prob by round
df['Win_Prob_weighted'] = [w*(2**(r-1)) for w,r in zip(df['Win_Prob'], df['Round'])]
df['Win_Prob_glm_weighted'] = [w*(2**(r-1)) for w,r in zip(df['Win_Prob_glm'], df['Round'])]
df['Win_Prob_gbm_weighted'] = [w*(2**(r-1)) for w,r in zip(df['Win_Prob_gbm'], df['Round'])]

df.head()

,Season,Slot,StrongSeed,WeakSeed,Round,TeamID_Strong,TeamName_Strong,TeamID_Weak,TeamName_Weak,TeamID_Winner,...,Win_Prob,RPI_Strong,RPI_Weak,RPI_Strong_Dummy,RPI_Weak_Dummy,Win_Prob_glm,Win_Prob_gbm,Win_Prob_weighted,Win_Prob_glm_weighted,Win_Prob_gbm_weighted
0,2018,R1W1,W01,W16,1,1437,Villanova,1347,Radford,1437,...,0.992647,2,128,1,1,0.948936,0.968941,0.992647,0.948936,0.968941
1,2018,R1W1,W01,W16,1,1437,Villanova,1347,Radford,1347,...,0.007353,2,128,1,1,0.948936,0.968941,0.007353,0.948936,0.968941
2,2018,R2W1,R1W1,R1W8,2,1437,Villanova,1439,Virginia Tech,1437,...,0.811594,2,61,1,1,0.796519,0.869192,1.623188,1.593037,1.738384
3,2018,R2W1,R1W1,R1W8,2,1437,Villanova,1439,Virginia Tech,1439,...,0.188406,2,61,1,1,0.796519,0.869192,0.376812,1.593037,1.738384
4,2018,R2W1,R1W1,R1W8,2,1347,Radford,1439,Virginia Tech,1347,...,0.254310,128,61,1,1,0.173994,0.223140,0.508621,0.347989,0.446279


In [ ]:
opt_results = optimize_tourney(df, 'Win_Prob_gbm_weighted', 'max')

In [ ]:
opt_results.head()

In [ ]:
opt_results.to_csv('../data/opt_results.csv', index=False)

In [ ]:
df.to_csv('../data/opt_input_v2.csv', index=False)